In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import cvxpy as cp
from scipy.stats import poisson, uniform, expon, pareto
from scipy.optimize import minimize, fsolve
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex

# tullock alpha < 1

In [151]:
b1, b2, v1, v2 = sp.symbols('b_1 b_2 v_1 v_2')

a = 1/2

x1 = b1**a / (b1**a + b2**a)
x2 = b2**a / (b1**a + b2**a)

u1 = v1 * x1 - b1
u2 = v2 * x2 - b2
u2

b_2**0.5*v_2/(b_1**0.5 + b_2**0.5) - b_2

In [152]:
part1 = sp.simplify(sp.diff(u1, b1))
part2 = sp.simplify(sp.diff(u2, b2))
part2

0.5*b_1**0.5*v_2/(b_2**0.5*(b_1**0.5 + b_2**0.5)**2) - 1

In [177]:
func1 = sp.lambdify([b1, b2], part1.evalf(subs={v1:11.}))
func2 = sp.lambdify([b1, b2], part2.evalf(subs={v2:1.}))

In [178]:
def eqs(p):
    b1, b2 = p
    return [func1(b1, b2), func2(b1, b2)]

sol = fsolve(eqs, (0.49, 0.12))
sol, sol[0]/sol[1], sum(sol)

(array([0.97897236, 0.08899749]), 10.99999999989627, 1.0679698490599445)

# monotonicity 
 
- we noticed that in the gamma-k construction, there was some monotonicity
  - payments monotone decreasing in k
  - welfare cost monotone increasing in k
- intuitively it feels like neither of these hold more generally, would be good to show that.

In [ ]:
# payments. there must be a new player that has a "high enough" true cost to make us pay more

In [202]:
def getEquilBids(cs, a, guess=None):
    n = len(cs)
    bs = [sp.Symbol('b{}'.format(i)) for i in range(n)]
    
    denom = sum([bi**(-a) for bi in bs])
    xs    = [bi**(-a)/denom for bi in bs]
    us    = [(bs[i] - cs[i])*xs[i] for i in range(n)]
    diffs = [sp.diff(us[i], bs[i]) for i in range(n)]
    funcs = [sp.lambdify(bs, diffs[i]) for i in range(n)]
    
    def eqs(bs):
        return [f(*bs) for f in funcs]
    
    initial_guess = cs
    if guess != None:
        initial_guess = guess
    
    return fsolve(eqs, initial_guess)

def getAllos(bids, a):
    denom = sum([b**(-a) for b in bids])
    return np.array(bids**(-a)) / denom

In [233]:
bids  = getEquilBids([4,2], a=3)
allos = getAllos(bids, 3)
bids, allos, np.dot(bids, allos)

(array([9.66287915, 8.8109329 ]),
 array([0.43121518, 0.56878482]),
 9.178305051291098)

In [274]:
bids = getEquilBids([4,2,20], a=3, guess=[9,8,30])
allos = getAllos(bids, 3)
bids, allos, np.dot(bids, allos)

(array([ 9.44715449,  8.56976702, 30.19759448]),
 array([0.42189055, 0.56519173, 0.01291772]),
 9.219310678501467)

In [285]:
# welfare. there may be a situation where adding a second "low cost" bidder actually improves welfare loss

In [282]:
costs = [4,2]
bids  = getEquilBids(costs, a=3)
allos = getAllos(bids, 3)
bids, allos, np.dot(costs, allos)

(array([9.66287915, 8.8109329 ]),
 array([0.43121518, 0.56878482]),
 2.862430353259135)

In [305]:
costs = [4,2,2]
bids  = getEquilBids(costs, a=3)
allos = getAllos(bids, 3)
bids, allos, np.dot(costs, allos)

(array([6.60620491, 4.73000037, 4.73000037]),
 array([0.15506709, 0.42246646, 0.42246646]),
 2.3101341775411135)

# welfare degredation in n

- we want to make statements like, given an alpha, the welfare degrades slowly in n even in the worst case
- consider the setting where c_1=1 and we add more players. the welfare decreases as you allocate to more people. 
- the best case is that the welfare loss is C/alpha, because you allocate 1-1/a to c_1=1 and 1/a to a player with value C. then your welfare is 1-1/a+C/a = (c+a-1)/a

In [317]:
(40+3-1)/3

14.0

In [352]:
costs = [1,2]
bids  = getEquilBids(costs, a=3)
allos = getAllos(bids, 3)
loss  = np.dot(costs, allos)
lossB = np.dot(costs, [2/3, 1/3])
bids, allos, loss, lossB, (loss-lossB)/loss

(array([4.40546645, 4.83143957]),
 array([0.56878482, 0.43121518]),
 1.4312151766295673,
 1.3333333333333333,
 0.0683907248152164)